# The Fermi Paradox

If there are so many stars with habitable planets, and if the origin and evolution of life is common and likely to develop into intelligent technoloically advanced civilisations that are capable of interstellar travel then why aren't they already here on Earth?

This has become known as the Fermi Paradox since Enrico Fermi famously remarked on it while at lunch with his colleagues. It was in a paper in 1975 by Michael Hart [Explanation for the Absence of Extraterrestrials on Earth. Quarterly Journal of the Royal Astronomical Society 1975, 16:128.](https://adsabs.harvard.edu/full/1975QJRAS..16..128H) that the arguments were formally examined. Hart concluded that if there were thousands of spacefaring extraterrestrial civilisations they would have had plenty of time to colonise the entire Milky Way and would therefore be here already. Since they are not here, he argued, they do not exist. He rejected all alternative arguments for why they may not be here (it is too far, too expensive, too difficult, they don't want to explore etc.) You can read my notes on this paper here:

[Why are extraterrestrials not on Earth?](https://open-research.gemmadanks.com/literature/why-extraterrestrials-are-not-on-earth/)

This notebook is an exploration and extension of Hart's calculations for the time it would take a civilisation to colonise the galaxy. He argued that it would be possible within two million years but he also assumed a civilisation would not die out (i.e. they have a longevity of over 2 million years). Earlier papers argued for much lower longevities. Hart dismissed the argument that civilisations would wipe themselves out before they managed to leave their planet, arguing that because humanity hasn't done that we should not expect the opposite of extraterrestrials. With today's compute power it is possible to build a simulation to test how longevity affects space colonisation and in this notebook I attempt a simple model of this.

## Assumptions
- The first civilisation to develop interstellar travel will begin to colonise the galaxy
- Other civilisations in the galaxy are irrelevant
- Only main sequence stars of spectral types G through to K are candidates for the emergence of ETI and for colonisation since they are most likely to host suitable planets
- Only a fraction of candidate stars have a habitable planet and the ETI will be able to evaluate this prior to sending settlers
- All candidate stars are equally likely to host the first civilisation
- The closest `max_num_missions` candidate stars within a radius `max_travel_distance` of the host star will be chosen as target stars
- Interstellar travel speed is fixed for all missions
- If a star is targetted its planet will be colonised with probability (`mission_success_rate`) once the ETI reach it and, if successful, secondary missions will begin after `mission_planning_time` years to uncolonised nearby stars following the same strategy as the initial mission
- Time to colonisation following mission launch is `travel_speed`/`distance_to_star` (there are never any delays)
- Resources for missions are always available and unlimited
- Missions continue until every reachable target star is colonised or all colonies die out
- A target star with a failed colony is not re-colonised
- The rotational velocity of the stars around the galactic centre is not relevant for colonisation
- The relativistic effects of space travel can be ignored since it is only relevant for the passengers, the journey time is computed relative to an outside observer

## Variables
- Longevity of a colony (years): `longevity`
- Speed of space travel (fraction of light speed): `travel_speed`
- Mission success rate `mission_success_rate`
- Time between missions (years): `mission_planning_time`
- Fraction of planets that are habitable: `fraction_planets_habitable`
- Maximum number of missions per colony: `max_num_missions`
- Maximum search radius for target stars (light years): `max_travel_distance` 

## Data requirements
Positions of all main sequence stars with spectral types G through to K.

This notebook uses a scaled-down rough approximation of the Milky Way to generate galaxies.

A more accurate model of the galaxy could be done using the most accurate map of the Milky Way so far from the [Gaia data release 3 archive](https://www.cosmos.esa.int/web/gaia/dr3). The Python package [astroquery](https://astroquery.readthedocs.io/en/latest/) can be used to extract relevant data. It contains data for the closest 1.4 billion stars out of 100 - 400 billion.



In [86]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import dependencies

In [87]:
import math
import numpy as np
import pandas as pd
import plotly.express as px

from colonisation import (scale_milky_way, generate_galaxy, plot_galaxy, evolve_eti,
                          colonise_galaxy)


## Set random seeds

In [88]:
galaxy_seed = 2
eti_seed = 1

## Initialise variables

In [103]:
longevity = 2000000
max_num_missions = None # 100
mission_planning_time = 0
max_travel_distance = 100
travel_speed = 0.1
mission_success_rate = 1.0 

## Generate galaxy

In [104]:
galaxy_radius, galaxy_thickness, num_target_stars = scale_milky_way(num_stars_to_model=1000)
#galaxy_radius, galaxy_thickness, num_target_stars = scale_milky_way(num_stars_to_model=None)

There are around 1750000000 habitable star systems in the Milky Way
Scaling galaxy to host 1000 stars...
Reducing volume of Milky Way by a factor of 1750000


In [105]:
star_map = generate_galaxy(num_target_stars,
                           galaxy_radius,
                           galaxy_thickness,
                           seed=galaxy_seed)

Generating galaxy with radius 438.5633228840598 and thickness 35.26762766806536 containing 1000 stars...


## Evolve ETI

In [106]:
star_map, eti_distance_from_galactic_centre = evolve_eti(seed=eti_seed, star_map=star_map)

ETI evolves on a star 40.88303661336448 from galactic centre (-32.0952449133936, -24.19199229333756, 7.487686261995693)


## Plot initial state

In [107]:
if num_target_stars < 1e5:
    plot_galaxy(star_map).show()
else:
    plot_galaxy(star_map.sample(10000)).show()

## Colonise the galaxy

1. For each colonised star
   - Find nearest neighbours
   - Launch missions


In [108]:
star_map = colonise_galaxy(star_map, max_travel_distance, travel_speed, max_num_targets=max_num_missions)

Finding neighbours for 1 newly colonised star
459 new missions launched from newly emerged ETI!
1 stars reached by ETI in year 54
0 missions failed
Finding neighbours for 1 newly colonised star
7 new missions launched in year 54!
1 stars reached by ETI in year 72
0 missions failed
Finding neighbours for 1 newly colonised star
7 new missions launched in year 72!
1 stars reached by ETI in year 92
0 missions failed
Finding neighbours for 1 newly colonised star
8 new missions launched in year 92!
1 stars reached by ETI in year 103
0 missions failed
Finding neighbours for 1 newly colonised star
6 new missions launched in year 103!
1 stars reached by ETI in year 113
0 missions failed
Finding neighbours for 1 newly colonised star
7 new missions launched in year 113!
1 stars reached by ETI in year 124
0 missions failed
Finding neighbours for 1 newly colonised star
10 new missions launched in year 124!
1 stars reached by ETI in year 126
0 missions failed
Finding neighbours for 1 newly colonised

In [113]:
colonisation_timespan = int(star_map['year_of_colonisation'].max())
num_unexplored_stars = len(star_map[star_map['state'] == 'unknown'])
print(f"Colonisation took {colonisation_timespan} years and left {num_unexplored_stars} stars unexplored")

Colonisation took 5567 years and left 6 stars unexplored


In [114]:
if num_target_stars < 1e5:
    plot_galaxy(star_map).show()
else:
    plot_galaxy(star_map.sample(10000)).show()

In [115]:
px.histogram(star_map, x='year_of_colonisation')

In [116]:
px.histogram(star_map[star_map['state'] == 'colonised'], x="colony_age")